# Proyecto 1
El Fondo de Población de las Naciones Unidas (UNFPA), en el marco de la Agenda 2030, busca 
aprovechar las opiniones ciudadanas para identificar problemáticas relacionadas con los 
Objetivos de Desarrollo Sostenible (ODS). En particular, se requiere un sistema capaz de analizar 
y clasificar textos en tres categorías:

- **ODS 1**: Fin de la pobrez
- **ODS 3**: Salud y bienestar
- **ODS 4**: Educación de calidad

El reto consiste en desarrollar un modelo de analítica de textos que reciba como entrada una 
opinión en lenguaje natural y prediga automáticamente a cuál ODS pertenece. Este modelo 
debe ser lo suficientemente robusto para manejar textos largos, con ruido (referencias, 
paréntesis, números) y en idioma español.

## 0. Reflexión sobre el proceso y conocimientos requeridos

Para resolver este caso, se debe seguir un proceso típico de **analítica de textos**:

1. **Entendimiento del negocio y los datos**  
   Comprender el contexto (ODS) y explorar la distribución de las etiquetas.

2. **Preprocesamiento de textos**  
   Normalización, limpieza de ruido, tokenización, eliminación de stopwords, lematización 
   en español y construcción de bigramas relevantes.

3. **Modelado**  
   Probar al menos tres algoritmos de clasificación supervisada (Naive Bayes, SVM, Random Forest, 
   redes neuronales, etc.), comparando métricas.

4. **Evaluación y resultados**  
   Seleccionar el mejor modelo, justificar métricas y explicar qué palabras o frases 
   llevan a la clasificación en cada ODS.

5. **Automatización y despliegue (Etapa 2)**  
   Construir un pipeline reproducible, crear una API REST y desplegar una aplicación web/móvil.

### Conocimientos requeridos:
- Manejo de librerías de NLP en Python (`nltk`, `spacy`, `scikit-learn`).
- Técnicas de limpieza de texto en español.
- Algoritmos de clasificación supervisada.
- Conceptos básicos de APIs y despliegue en aplicaciones.

### Dudas actuales:
- ¿Qué técnica de vectorización será más efectiva: TF-IDF o embeddings?
- ¿Qué tan balanceadas están las clases (ODS 1, 3, 4)?
- ¿Cuánta aumentación de datos será necesaria para mejorar la clase minoritaria?



### 1. Instalación e importanción de librerías.


In [1]:
# --- Instalación (mac) ---
# !python3 -m pip install -U nltk spacy scikit-learn matplotlib seaborn unidecode
# !python3 -m spacy download es_core_news_md

# --- Imports base ---
import re, string, unicodedata
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
# Si quisieras stemming en español (no recomendado para el pipeline final):
# from nltk.stem import SnowballStemmer

import spacy
# Carga del modelo de spaCy en español (md tiene vectores; lg si quieres mejores)
nlp = spacy.load("es_core_news_md")

# Utilidad: solo úsalo DESPUÉS de lematizar si quieres una segunda versión sin acentos
import unidecode

# Sklearn: vectorización y modelos
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix

# NLTK data 
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

# Stopwords en español 
STOP_ES = set(stopwords.words('spanish'))
